# Predicting NYC apartment's value with open data

According to real estate brokers in NYC, there are a variety of factors they consider to determine the value of your apartment, not limited to

1. Recent sales in your building / Neighbourhood
2. Square footage
3. Renovation status
4. View, close to subway, # of bedrooms etc. 

Unfortunately this data, especially apt sq footage, is not easily available for non REBNY members. 

In this notebook, I will explore an alternative approach to price Manhattan apartments. 

In [1]:
import numpy as np
import pandas as pd

from datascience import *

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import locale
import os
import xlrd as xlrd

## Import data

Source
1. [Annualized Sales Data](https://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page)
2. [Rolling Sale data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [2]:
# Constants
raw_directory = "data/raw/"
csv_directory = "data/csv/"

# Retrieve current working directory (`cwd`)
cwd = os.getcwd()

# List data files and directories in current directory
excel_files = os.listdir(raw_directory)

# Select only tje xls files
excel_files = [k for k in excel_files if '.xls' in k]

In [3]:
excel_files[:len(excel_files)]

['2015_manhattan.xls',
 '2011_manhattan.xls',
 '2016_manhattan.xls',
 '2012_manhattan.xls',
 'rollingsales_manhattan.xls',
 'sales_manhattan_03.xls',
 'sales_manhattan_06.xls',
 'sales_manhattan_04.xls',
 'sales_2007_manhattan.xls',
 'sales_manhattan_05.xls',
 '2009_manhattan.xls',
 '2013_manhattan.xls',
 '2017_manhattan.xls',
 'sales_2008_manhattan.xls',
 '2010_manhattan.xls',
 '2014_manhattan.xls']

Unfortunately not all files are formatted the same was. Some have the header in row 4, others in row 5. We can check by making sure 'BOROUGH' is the first column in the imported dataset 

In [4]:

# Create an data store
all_sales_data = pd.DataFrame()

# Load individual excel files. 
for excel_file in excel_files:
    print(excel_file)
    
    # Read excel, Note the headers could in row 4 or row 5 (index=3 or 4). 
    yearly_sales_data = pd.read_excel(raw_directory+excel_file, header=3, encoding='sys.getfilesystemencoding()')
   
    # Check if the first column is "BOROUGH"
    if not yearly_sales_data.columns[0].startswith('BOROUGH'):
        # Otherwise the data starts from row 5.
         yearly_sales_data = pd.read_excel(raw_directory+excel_file, header=4, encoding='sys.getfilesystemencoding()')
    
    print(yearly_sales_data.shape)
    
    yearly_sales_data.rename(columns=lambda x: x.strip(), inplace=True)
    
    all_sales_data = all_sales_data.append(yearly_sales_data)
    
    print(all_sales_data.shape)





2015_manhattan.xls
(24989, 21)
(24989, 21)
2011_manhattan.xls
(21500, 21)
(46489, 21)
2016_manhattan.xls
(21241, 21)
(67730, 21)
2012_manhattan.xls
(26258, 21)
(93988, 21)
rollingsales_manhattan.xls
(16828, 21)
(110816, 21)
sales_manhattan_03.xls
(22210, 21)
(133026, 21)
sales_manhattan_06.xls
(26352, 21)
(159378, 21)
sales_manhattan_04.xls
(25894, 21)
(185272, 21)
sales_2007_manhattan.xls
(28439, 21)
(213711, 21)
sales_manhattan_05.xls
(26388, 21)
(240099, 21)
2009_manhattan.xls
(19166, 21)
(259265, 21)
2013_manhattan.xls
(26715, 21)
(285980, 21)
2017_manhattan.xls
(18642, 21)
(304622, 23)
sales_2008_manhattan.xls


/Users/somya/Library/Python/3.7/lib/python/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(25994, 21)
(330616, 23)
2010_manhattan.xls
(17296, 21)
(347912, 23)
2014_manhattan.xls
(24524, 21)
(372436, 23)


Spot check to verify data

In [5]:

all_sales_data.sample(10)


,ADDRESS,APARTMENT NUMBER,BLOCK,BOROUGH,BUILDING CLASS AS OF FINAL ROLL 17/18,BUILDING CLASS AT PRESENT,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,COMMERCIAL UNITS,EASE-MENT,...,NEIGHBORHOOD,RESIDENTIAL UNITS,SALE DATE,SALE PRICE,TAX CLASS AS OF FINAL ROLL 17/18,TAX CLASS AT PRESENT,TAX CLASS AT TIME OF SALE,TOTAL UNITS,YEAR BUILT,ZIP CODE
935,305 WEST 18 STREET,1G,742,1,NaN,R4,R4,13 CONDOS - ELEVATOR APARTMENTS,0,,...,CHELSEA,1,2006-10-04,400000,NaN,2,2,1,1950,10011
21727,170 EAST 87TH STREET,W18H,1515,1,NaN,R4,R4,13 CONDOS - ELEVATOR APARTMENTS,0,,...,UPPER EAST SIDE (79-96),1,2012-10-24,1200000,NaN,2,2,1,0,10128
17255,459 WASHINGTON STREET,7N,595,1,NaN,R4,R4,13 CONDOS - ELEVATOR APARTMENTS,0,,...,SOHO,1,2006-04-03,2000000,NaN,2,2,1,0,10013
22876,37 WEST 89TH STREET,,1203,1,NaN,C5,C5,07 RENTALS - WALKUP APARTMENTS,0,,...,UPPER WEST SIDE (79-96),10,2015-06-15,5200000,NaN,2B,2,10,1930,10024
18227,"944 FIFTH AVENUE, 4",,1390,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,UPPER EAST SIDE (59-79),0,2006-12-04,7500000,NaN,2,2,0,1925,10021
1455,"415 WEST 57TH STREET, 3C",,1067,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,CLINTON,0,2004-05-14,250000,NaN,2,2,0,1900,10019
21265,"300 EAST 85TH STREET, 201",,1547,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,UPPER EAST SIDE (79-96),0,2012-11-08,510000,NaN,2,2,0,1987,10028
10294,200 WEST 56TH STREET,,1027,1,NaN,RH,RH,,0,,...,MIDTOWN WEST,0,2012-12-03,0,NaN,4,4,1,0,10019
13778,200 WEST 56TH STREET,,1027,1,NaN,RH,RH,45 CONDO HOTELS,0,,...,MIDTOWN WEST,0,2013-12-30,100,NaN,4,4,1,0,10019
4785,"245 E 24TH STREET, 15G",,905,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,GRAMERCY,0,2008-11-03,0,NaN,2,2,0,1963,10010


In [6]:
# Check for duplicate entries
sum(all_sales_data.duplicated(all_sales_data.columns))



16137

In [7]:
#Delete the duplicates and check that it worked
all_sales_data = all_sales_data.drop_duplicates(all_sales_data.columns, keep='last')
sum(all_sales_data.duplicated(all_sales_data.columns))

0

Save the data as a CSV for further clean up and analyis. See Step 2. 

In [8]:
# save to csv
all_sales_data.to_csv(csv_directory+"manhattan.csv")